In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd drive/MyDrive/univer/Expociencia/Code/Electra

/content/drive/MyDrive/univer/Expociencia/Code/Electra


In [3]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 34.4 MB/s 
     |████████████████████████████████| 6.6 MB 53.2 MB/s 
     |████████████████████████████████| 86 kB 6.8 MB/s 
     |████████████████████████████████| 596 kB 60.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [4]:
import torch
import pandas as pd
import numpy as np

from torch.utils.data import Dataset, DataLoader
import gdown
from transformers import ElectraForSequenceClassification, ElectraTokenizerFast
from sklearn.metrics import accuracy_score, f1_score
from torch import nn, optim
from pathlib import Path

In [5]:
model = ElectraForSequenceClassification.from_pretrained('mrm8488/electricidad-base-discriminator', num_labels = 5)
tokenizer = ElectraTokenizerFast.from_pretrained('mrm8488/electricidad-base-discriminator', do_lower_case=False)

Downloading:   0%|          | 0.00/467 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/419M [00:00<?, ?B/s]

Some weights of the model checkpoint at mrm8488/electricidad-base-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.weight']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at mrm8488/electricidad-base-discriminator and are newly initialized: ['

Downloading:   0%|          | 0.00/236k [00:00<?, ?B/s]

In [6]:
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")

In [ ]:
#tokenizer.convert_tokens_to_ids('[PAD]') #[SEP] -> 5, [CLS].-> 4, [PAD]-> 1, [MASK]-> 0

1

In [7]:
#train = pd.read_csv('train_yelp_clean.csv')
#val = pd.read_csv('val_yelp_clean.csv')
test = pd.read_csv('test_yelp_clean.csv')

In [8]:
#train['Score'] = train['Score'].apply(lambda x: x-1) 
#val['Score'] = val['Score'].apply(lambda x: x-1) 
test['Score'] = test['Score'].apply(lambda x: x-1)

In [9]:
MAX_LEN = 128 #Fijo
BATCH_SIZE = 32
NCLASSES= 5 #Fijo
DropOut = 0.1
EPOCHS= 1
LEARNING_RATE=2e-5
with_pretrain = False
#NAME = 'electricidad-base-discriminator'
#if with_pretrain:
#    NAME = NAME+"_pretrain"
#    discriminador = ElectraForPreTraining.from_pretrained('mrm8488/electricidad-base-discriminator')

In [ ]:
if with_pretrain:
    ckp = torch.load('Pretrined_ELECTRAGENERATOR.pth', map_location=device)
    discriminador.load_state_dict(ckp['weight_model'])
    model.electra.load_state_dict(discriminador.electra.state_dict())

In [ ]:
myfile = Path(f'{NAME} LR {LEARNING_RATE}.txt')
myfile.touch(exist_ok=True)

In [10]:
class LoadDataset(Dataset):
    def __init__(self, reviews, labels, tokenizer, max_len, mode_truncation):
        self.reviews = reviews
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.mode_truncation = mode_truncation

    def __len__(self):
        return len(self.reviews)
    
    def __getitem__(self, item):
        review = str(self.reviews[item]) 
        label = self.labels[item]
        tokens = self.tokenizer.tokenize(review) 
        tokens = ['[CLS]'] + tokens + ['[SEP]']
        if len(tokens) < self.max_len:
            tokens = tokens + ['[PAD]' for item in range(self.max_len-len(tokens))]
        elif len(tokens) > self.max_len:
            if self.mode_truncation == "head":
                tokens = tokens[:self.max_len-1] + ['[SEP]']    
            elif self.mode_truncation == "head+tail":
                tokens = tokens[:65] + tokens[-63:]
            elif self.mode_truncation == "tail":
                tokens = ['[CLS]'] + tokens[-int((self.max_len)/2):]
        token_ids = self.tokenizer.convert_tokens_to_ids(tokens)
        tokens_ids_tensor = torch.tensor(token_ids)
        attn_mask = (tokens_ids_tensor != 1).long()
        return{
            'review':review,
            'input_ids':tokens_ids_tensor.flatten(),
            'attention_mask':attn_mask.flatten(),
            'label':torch.tensor(label, dtype=torch.long)}

def data_loader(df, tokenizer, max_len, batch_size, modo):
  dataset = LoadDataset(
    reviews=df.Text.to_numpy(),
    labels = df.Score.to_numpy(),
    tokenizer=tokenizer,
    max_len=max_len,
    mode_truncation = modo
  )
  return DataLoader(dataset, batch_size= BATCH_SIZE, num_workers=2)

In [11]:
#train_data_loader = data_loader(train, tokenizer, MAX_LEN, BATCH_SIZE, "head+tail")
#validation_data_loader = data_loader(val, tokenizer, MAX_LEN, BATCH_SIZE, "head+tail")
test_data_loader = data_loader(test, tokenizer, MAX_LEN, BATCH_SIZE, "head+tail")

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [12]:
model.to(device)
print('ok')

ok


In [13]:
def train_mode(model , epoch, train_data_loader, optim):
    model = model.train()
    train_loop = tqdm(train_data_loader, leave=True, position = 0)
    losses = []
    accuracy_global = []
    f1_score_global = []
    f1_weighted_global = []
    i = 0
    for batch in train_loop:
        optim.zero_grad()
        input_ids= batch['input_ids'].to(device)
        attention_mask= batch['attention_mask'].to(device)
        labels= batch['label'].to(device)
        outputs = model(input_ids, 
                    attention_mask=attention_mask, 
                    labels=labels
                    )
        logit = outputs.logits
        loss = outputs.loss
        _, preds = torch.max(outputs.logits, dim=1)
        ac = accuracy_score(preds.cpu().clone().detach().numpy(), labels.cpu().clone().detach().numpy())
        f1 = np.mean(f1_score(preds.cpu().clone().detach().numpy(), labels.cpu().clone().detach().numpy(), average=None))
        f1_weight = np.mean(f1_score(preds.cpu().clone().detach().numpy(), labels.cpu().clone().detach().numpy(), average='weighted'))
        f1_weighted_global.append(f1_weight)
        accuracy_global.append(ac)
        f1_score_global.append(f1)
        losses.append(loss.item())
        loss.backward()
        optim.step()
        train_loop.set_description(f'epoch  {epoch}')
        train_loop.set_postfix(loss=np.mean(losses), acc = np.mean(accuracy_global), f1 = np.mean(f1_score_global), f1_weight = np.mean(f1_weighted_global))
        
        if i%1000==0 and i>9999:
            val_loop = tqdm(validation_data_loader, leave=True, position = 0)
            meanloss, meanacc, meanf1, meanweight, history = eval_mode(model, epoch, val_loop) 
            with open (f'{NAME} LR {LEARNING_RATE}.txt','a') as f:
                f.write(f'''    name: {NAME}
                Iteracion : {i}
                Train:
                    -loss: {np.mean(losses)}, 
                    mean: Accuracy: {np.mean(accuracy_global)} -- f1: {np.mean(f1_score_global)}-- f1_weighted : {np.mean(f1_weighted_global)}
                    Var : acc: {np.std(accuracy_global)**2} -- f1: {np.std(f1_score_global)**2}--f1_weighted : {np.std(f1_weighted_global)**2}
                Validation
                    -loss: {meanloss},
                     mean: Accuracy: {meanacc} -- f1: {meanf1}, f1_weight : {meanweight} 
                     Var : acc: {np.std(history['acc'])**2} -- f1: {np.std(history['f1'])**2} -- f1_weight: {np.std(history['f1_weight'])**2}
                ---------------------------------   
                
    ''')
            checkpoint = {'epoch': i, 'state_dict': model.state_dict()}  
            torch.save(checkpoint, f'Electra_{epoch}_{i}.pth')
        i+=1
    return np.mean(losses), np.mean(accuracy_global), np.mean(f1_score_global), np.mean(f1_weighted_global), {
        'loss':losses,
        'acc': accuracy_global,
        'f1': f1_score_global,
        'f1_weight':f1_weighted_global
    }
def eval_mode(model, epoch, loop):
    losses = []
    accuracy_global = []
    f1_score_global = []
    f1_weighted_global = []
    model = model.eval()
    with torch.no_grad():
        for batch in loop:
            input_ids= batch['input_ids'].to(device)
            attention_mask= batch['attention_mask'].to(device)
            labels= batch['label'].to(device)
            outputs= model(input_ids, 
                    attention_mask=attention_mask, 
                    labels=labels
                    )
            logit = outputs.logits
            _, preds = torch.max(outputs.logits, dim=1)
            ac = accuracy_score(preds.cpu().clone().detach().numpy(), labels.cpu().clone().detach().numpy())
            f1 = np.mean(f1_score(preds.cpu().clone().detach().numpy(), labels.cpu().clone().detach().numpy(), average=None))
            f1_weight = np.mean(f1_score(preds.cpu().clone().detach().numpy(), labels.cpu().clone().detach().numpy(), average='weighted'))
            accuracy_global.append(ac)
            f1_score_global.append(f1)
            f1_weighted_global.append(f1_weight)
            losses.append(outputs.loss.item())
            loop.set_description(f'epoch  {epoch}')
            loop.set_postfix(loss=np.mean(losses), acc = np.mean(accuracy_global), f1 = np.mean(f1_score_global), f1_weight=np.mean(f1_weighted_global))
    return np.mean(losses), np.mean(accuracy_global), np.mean(f1_score_global), np.mean(f1_weighted_global), {
        'loss':losses,
        'acc':accuracy_global,
        'f1':f1_score_global,
        'f1_weight':f1_weighted_global
    }

In [14]:
from tqdm.auto import tqdm
import warnings
warnings.filterwarnings('ignore')

In [ ]:
for epoch in range(EPOCHS):
    val_loop = tqdm(validation_data_loader, leave=True, position = 0)
    train_loss, train_acc, train_f1, train_f1_weight, train_history = train_mode(model , epoch, train_data_loader, optimizer)
    checkpoint = {'epoch': epoch + 1, 'state_dict': model.state_dict(),
              'optimizer': optimizer.state_dict()}  
    torch.save(checkpoint, f'checkpoint_Electra_{epoch+1}.pth')             
    meanloss, meanacc, meanf1, mean_f1_weight, history = eval_mode(model, epoch, val_loop)
    with open (f'{NAME} LR {LEARNING_RATE}.txt','a') as f:
            f.write(f'''    name: {NAME}
            epoch : {epoch+1}
            Train
                -loss: {train_loss},
                mean : Accuracy: {train_acc} -- f1: {train_f1}-- f1_Weight = {train_f1_weight}
                Var : acc: {np.std(train_history['acc'])**2} -- f1: {np.std(train_history['f1'])**2}, f1_weight: {np.std(train_history['f1_weight'])**2}

            Validation
                    -loss: {meanloss},
                     mean: Accuracy: {meanacc} -- f1: {meanf1}, f1_weight : {mean_f1_weight} 
                     Var : acc: {np.std(history['acc'])**2} -- f1: {np.std(history['f1'])**2} -- f1_weight: {np.std(history['f1_weight'])**2}
                ----------------------------------   
    ----            
    ''')           

  0%|          | 0/767 [00:00<?, ?it/s]

  0%|          | 0/19915 [00:00<?, ?it/s]

In [15]:
checkpoint = torch.load('Electra_0_16000.pth')
checkpoint.keys() 

dict_keys(['epoch', 'state_dict'])

In [16]:
model.load_state_dict(checkpoint['state_dict'])

<All keys matched successfully>

In [18]:
test_loop = tqdm(test_data_loader, leave=True, position = 0)
meanloss, meanacc, meanf1, meanweight, history = eval_mode(model, 0, test_loop) 

  0%|          | 0/767 [00:00<?, ?it/s]

In [20]:
print(meanacc, meanweight)

0.6664602346805737 0.6673061241474346


#Inference

In [ ]:
def clasificacion_review(model, review, tokenizer, mode_truncation):
    max_len = 128
    model.to('cpu')
    model = model.eval()
    tokens = tokenizer.tokenize(review) 
    tokens = ['[CLS]'] + tokens + ['[SEP]']
    if len(tokens) < max_len:
        tokens = tokens + ['[PAD]' for item in range(max_len-len(tokens))]
    elif len(tokens) > max_len:
        if mode_truncation == "head":
            tokens = tokens[:max_len-1] + ['[SEP]']    
        elif mode_truncation == "head+tail":
            tokens = tokens[1:64] + tokens[-64:-1]#int((self.max_len)/2)
            tokens = ['[CLS]'] + tokens + ['[SEP]']
        elif mode_truncation == "tail":
            tokens = ['[CLS]'] + tokens[-int((max_len)/2):]
    token_ids = tokenizer.convert_tokens_to_ids(tokens)
    tokens_ids_tensor = torch.tensor(token_ids).flatten().to('cpu')
    input_ids = torch.unsqueeze(tokens_ids_tensor, dim=0)
    inputs = tokenizer(review, return_tensors="pt")
    attn_mask = (tokens_ids_tensor != 1).long().flatten().to('cpu')
    attention_mask = torch.unsqueeze(attn_mask, dim=0).to('cpu')
    with torch.no_grad():
        output = model(input_ids, attention_mask)
        _, prediction = torch.max(output.logits,dim=1)
    return prediction.item()+1

In [ ]:
clasificacion_review(model, 'Esta muy bueno el cargador por ahí el tema del voltaje tiene algo pero aun así me convence',tokenizer,'head+tail')

4

In [ ]:
clasificacion_review(model, 'este muñeco de miko esta muy bonito tiene todos la figuara del personaje eso me encanta pero le pondre 4 estrellas porque era muy pequeño',tokenizer,'head+tail')

4

In [ ]:
clasificacion_review(model, 'la comida esta una horrible, soleado y seco eso no se puede en un restaurante de este nivel, si quieren mejor tienen que aprender  a cocinar',tokenizer,'head+tail')

1

In [ ]:
clasificacion_review(model, 'el "pato" no me gusto el nombre del producto pero aun asi me encanto cumplió con las expectitvas y encima me ayudo a ganar',tokenizer,'head+tail')

4

In [ ]:
clasificacion_review(model, 'los adufinos fueron inútiles aun puedo rescatar algunas cosas como la calidad del sonido pero el problema del bluetho hace que no pueda usarlo en cada video, 2 estrellas',tokenizer,'head+tail')

2

In [ ]:
clasificacion_review(model, 'los audifinos fueron inútiles casi pierdo mi listening pero por el precio era de esperarse',tokenizer,'head+tail')

2

In [ ]:
import csv
def inference_csv(name,df):
    fieldnames = ['Text', 'Y', 'Y^']
    with open(name, 'a', newline='') as csvfile:
        for i in range(len(df.Text)):
            prediction = clasificacion_review(model, df.Text[i], tokenizer, "head+tail")
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writerow({'Text':df.Text[i], 'Y':df.Score[i]+1, 'Y^': prediction})

In [ ]:
inference_csv('inference_test.csv', test)
inference_csv('inference_val.csv', val)